In [ ]:
import numpy as np
import plotly.graph_objects as go
import hyperspy.api as hs

In [ ]:
path = "../../../Masteroppgave/2023-03-08_EDS-Apreo/exports/"
s = hs.load(path + 'GaAs_30kV_50pA.emsa', signal_type='EDS_SEM')
elements = ['Ga', 'As']
s.set_elements(elements)

In [ ]:
noise_peak = s.isig[15:28].deepcopy()
noise_peak_channels = np.arange(15, 28)
np_kev = noise_peak.axes_manager[0].scale

In [ ]:
s.data[:33] = 0
x = s.axes_manager.signal_axes[0].axis
m = s.create_model()
m.fit()
'f'

In [ ]:
scale = s.axes_manager[0].scale
offset = s.axes_manager[0].offset

In [ ]:
def kev2channel(kev):
    return (kev - offset) / scale
def channel2kev(channel):
    return channel * scale + offset
def remove_low_values(y, threshold):
    y[y < threshold] = np.nan
    return y

In [ ]:
internal_Si = (noise_peak.data * 0.5) + 610
internal_Si_channels = noise_peak_channels + 175
internal_Si_x = np_kev + 1.74

In [ ]:
bg_channels = np.arange(0, 2048)
bg_kev = bg_channels * scale + offset
bg = m[0].function(bg_kev) * scale

In [ ]:
w = .6

e1 = 1.089
gaLa = np.arange(e1-w, e1+w, .004)
gaLa_channels = np.array([kev2channel(kev) for kev in gaLa])
e2 = 9.2517
gaKa = np.arange(e2-w, e2+w, .004)
gaKa_channels = np.array([kev2channel(kev) for kev in gaKa])


y_GaLa = (m['Ga_La'].function(gaLa) )* scale# + m[0].function(gaLa) 
y_GaKa = (m['Ga_Ka'].function(gaKa) )* scale

In [ ]:
y_GaLa = remove_low_values(y_GaLa, 435)
y_GaKa = remove_low_values(y_GaKa, 400)

In [ ]:
sum_peak = y_GaLa * 0.25 + 750
sum_peak_channels = gaLa_channels + kev2channel(e1)

In [ ]:
escape_peak = y_GaKa *0.12 + 550
escape_peak_channels = gaKa_channels - kev2channel(1.74)

In [ ]:
channel2kev(215)

In [ ]:
fig = go.Figure()
fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.995))
fig.update_layout(width=800,height=400,)
fig.update_layout(margin=dict(l=5, r=5, t=5, b=5))
channels = np.arange(0, 2048)
x=channels
y = s.data
y2 = m.as_signal().data

# fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='data', visible='legendonly'))
# fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', name='fit'))

fig.add_trace(go.Scatter(x=gaLa_channels, y=y_GaLa, mode='lines', name='Peak at E<sub>1</sub>'))
fig.add_trace(go.Scatter(x=gaKa_channels, y=y_GaKa, mode='lines', name='Peak at E<sub>2</sub>'))
fig.add_trace(go.Scatter(x=noise_peak_channels-13, y=noise_peak*1, mode='lines', name='Noise peak'))
fig.add_trace(go.Scatter(x=internal_Si_channels, y=internal_Si, mode='lines', name='Internal Si Kα'))
fig.add_trace(go.Scatter(x=sum_peak_channels, y=sum_peak, mode='lines', name='E<sub>1</sub> sum peak'))
fig.add_trace(go.Scatter(x=escape_peak_channels, y=escape_peak, mode='lines', name='E<sub>2</sub> escape peak'))
fig.add_trace(go.Scatter(x=bg_channels, y=bg, mode='lines', name='Background', line=dict(color='black')))


fig.update_xaxes(range=[-10, 1250])

fig.update_xaxes(title_text="Energy [keV]")#, showticklabels=False)
fig.update_yaxes(title_text="Intensity [counts]")#, showticklabels=False)


# annotations
# annotate E1-E(Si Ka) at 751 3000
fig.add_annotation(text="E<sub>1</sub>", x=130, y=47000, showarrow=False)
fig.add_annotation(text="Sum peak<br>2×E<sub>1</sub>", x=267, y=15500, showarrow=False)
fig.add_annotation(text="Escape peak<br>E<sub>2</sub>-E(Si Kα)", x=750, y=6300, showarrow=False)
# annotate E1 at 945 18k
fig.add_annotation(text="E<sub>2</sub>", x=945, y=19500, showarrow=False)
# annotate Si Ka at215 6500
fig.add_annotation(text="Stray<br>Si Kα", x=200, y=9300, showarrow=False)
# annotate 0 at 12 19k
fig.add_annotation(text="Noise<br>peak", x=35, y=14100, showarrow=False, )# font=dict(size=12))


# tickvals = [0, 200, 400, 600, 800, 1000, 1200, 1400]
# tickvals are -.2, 2, 4, 6, 8, 10, 12
tickvals = [kev2channel(kev) for kev in [2, 4, 6, 8, 10, 12]]
ticktext = [str(round(channel2kev(tickval),2)) for tickval in tickvals]
fig.update_xaxes(tickvals=[0] + tickvals, ticktext=['0'] + ticktext)
# show only tick 0 for y axis
# fig.update_yaxes(tickvals=[0], ticktext=['0'])

# fig.update_layout(yaxis_type='log')

fig

In [ ]:
fig.write_image('../figures/figure_artifacts.pdf')
fig.write_image('../../mastersthesis/figures/eds_artifacts.pdf')